In [ ]:
# установка внешних программ и библиотек на виртуальную машину
!git clone --recurse-submodules https://github.com/KarlsruheMIS/KaMIS.git
%cd KaMIS
!./compile_withcmake.sh

%cd ..


!pip3 install python-sat
#!sudo apt install minisat
!git clone https://github.com/arminbiere/kissat.git
%cd kissat
!./configure
!make

%cd ..

!cp KaMIS/deploy/redumis redumis
!cp KaMIS/deploy/online_mis online_mis
!cp kissat/build/kissat kissat1



Cloning into 'KaMIS'...
remote: Enumerating objects: 1449, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 1449 (delta 27), reused 21 (delta 19), pack-reused 1399
Receiving objects: 100% (1449/1449), 12.76 MiB | 13.65 MiB/s, done.
Resolving deltas: 100% (818/818), done.
/content/KaMIS
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test COMPILER_SUPPORTS_FUNROLL_LOOPS
--

In [1]:
# функции из библиотеки
# раскраска графа в формате  DIMACS, преобразование в CNF

import math
import numpy as np
from numpy import linalg as LA
import networkx as nx

#import matplotlib.pylab as plt

#from google.colab import files 
#files.upload() 
#files.download("file.txt")

#https://pysathq.github.io/
#!pip install python-sat
#import pysat
from pysat.formula import CNF
from pysat.solvers import *

import os
import sys
import random
from zipfile import ZipFile
#from tqdm import tqdm_notebook as tqdm
from itertools import combinations, permutations
import glob

out_dir = ""

class Utils:
    
    def read_dimacs_graph(file = 'graph.col'):
        
        if not (os.path.exists(file) and os.path.getsize(file) > 0):        
            raise Exception("File " + file + " not found")
        
        nodes = []    
        edges = []
        labels = []
        got_labels = False
        nnodes = nedges = 0
        
        with open(file, 'r') as f:
            for line in f:
                line = [l.strip() for l in line.split(' ')]
                if line[0] == 'c': #comment
                    continue
                elif line[0] == 'p': #problem
                    nnodes = int(line[2])
                    nedges = int(line[3])
                    nodes = list(range(1, nnodes + 1))
                    labels = [0] * nnodes
                elif line[0] == 'e': #edge
                    edges.append((int(line[1]), int(line[2])))
                elif line[0] == 'l':
                    labels[int(line[1]) - 1] = int(line[2])
                    got_labels = True

        if got_labels:        
            nodes = [(n, {'c' : l}) for n, l in zip(nodes, labels)]

        g = nx.Graph()
        g.add_nodes_from(nodes)
        g.add_edges_from(edges)
        return g

    def write_dimacs_graph(file = 'graph.col', g = nx.Graph(), comments = []):
        with open(file, 'w') as f:
            for c in comments:
                f.write("c " + c + "\n")
            f.write("p EDGE {} {}\n".format(g.number_of_nodes(), g.number_of_edges()))
            for u, v in g.edges():
                f.write("e {} {}\n".format(u, v))
            for node in g.nodes():
                if 'c' in g.node[node]:
                    f.write("l {} {}\n".format(node, g.node[node]['c']))

    def draw_with_colors(g = nx.Graph()):
        color_map = []
        for node in g.nodes():
            if 'c' in g.node[node]:
                color_map.append(g.node[node]['c'] * 10)            
        nx.draw(g, pos = nx.spring_layout(g, scale=2), node_color=color_map, with_labels=True, cmap = plt.cm.jet)

    def write_proof(file = "proof.txt", proof = []):
        with open(file, 'w') as f:
            for p in proof:
                f.write("%s\n" % str(p))

    def zip_files(file = "archive.zip", files = []):
        with ZipFile(file, 'w') as archive:
            for f in set(files):                
                if not (os.path.exists(file) and os.path.getsize(file) > 0):        
                    raise Exception("File " + file + " not found")
                archive.write(f)
                
def find_triangle(g = nx.Graph()):
    for a in g:
        for b, c in combinations(g[a], 2):
            if b in g[c]:
                return [a, b, c]
    return []
             
    #return set(frozenset([a, b, c]) for a in g for b, c in combinations(g[a], 2) if b in g[c])


def find_isolates(g = nx.Graph()):
    isolates = []
    for n in g:
        if g.degree(n) == 0:
            isolates.append(n)
    return isolates

class ColMap:
    
    def __init__(self, g = nx.Graph(), ncolors = 40):
        
        self.ncolors = ncolors
        self.cmap = dict()
        self.cunmap = dict()
    
        i = 1
        for node in g.nodes():
            for color in range(1, ncolors + 1):            
                self.cmap[(node, color)] = i
                self.cunmap[i] = (node, color)
                i += 1    

    def enc(self, node, color):
        return self.cmap[(node, color)]

    def dec(self, node_color):
        return self.cunmap[node_color]

class ColSAT:
   
    def __init__(self, g = nx.Graph(), ncolors = 10):
        
        self.ncolors = ncolors
        self.g = g.copy()
        self.cmap = ColMap(g, ncolors)        

    def check_coloring(self):
        for n1, n2 in self.g.edges():
            if 'c' not in self.g.node[n1] or 'c' not in self.g.node[n2]:
                return False
            if self.g.node[n1]['c'] == self.g.node[n2]['c']:
                return False
        return True
    
    def apply_model(self):
        
        check = set()
        for var in self.model[self.model > 0]:        
            node, color = self.cmap.dec(var)
            self.g.node[node]['c'] = color
            if (node, color) in check:
                raise Exception("Two colors for one node???")
            else:
                check.add((node, color))
        
        self.colored = self.check_coloring()
        
        if self.colored != self.solved:
            raise Exception("Something went wrong!")
        
        return self.colored
        
    def build_cnf(self, ):
        
        self.formula = CNF()
        colors = list(range(1, self.ncolors + 1))    

        maxc = -1        # condition k<12 is not for general case
        k = 0
        for clique in nx.find_cliques(self.g):
            if len(clique)>maxc:
                maxclique = clique.copy()  
                maxc = len(clique)
            if k<12:
                k += 1
                continue
           
            col = 1
            if len(maxclique)>self.ncolors:
                print("Size of the maximal clique is {}".format(len(maxclique)))
                raise Exception("Size of the maximal clique is {} > {}".format(len(maxclique),ncolors))
            for v in maxclique:
                self.formula.append([self.cmap.enc(v, col)])
                col += 1
            break

        for n1, n2 in self.g.edges():
            for c in colors:            
                self.formula.append([-self.cmap.enc(n1, c), -self.cmap.enc(n2, c)])


        for n in self.g.nodes():
            #if not n in specials:
            self.formula.append([self.cmap.enc(n, c) for c in colors])
            #for c1 in colors:
            #    for c2 in colors:
            #        if c1 < c2:
            #            self.formula.append([-self.cmap.enc(n, c1), -self.cmap.enc(n, c2)])
        
        return self.formula
    
    def solve_cnf(self, solver = ''):
        
        triangle = find_triangle(self.g)
        assumptions = []
        if len(triangle) > 0:            
            assumptions = [self.cmap.enc(triangle[0], 1), self.cmap.enc(triangle[1], 2), self.cmap.enc(triangle[2], 3)]
            
        #Glucose3, Glucose4, Lingeling, MapleChrono, MapleCM, Maplesat, Minisat22, MinisatGH
        #with Glucose4(bootstrap_with=self.formula.clauses, with_proof=True) as ms:        
        with Lingeling(bootstrap_with=self.formula.clauses) as ms:
            self.solved = ms.solve(assumptions=assumptions)
            if self.solved:
                self.model = np.array(ms.get_model())
                self.apply_model()
            else:                
                self.proof = []#ms.get_proof()
                self.colored = False
                
        return self.solved






In [2]:
# выбор графа на вершинах гиперкуба (покрышки), удаление независимого множества,
# проверка существования раскраски

import networkx as nx

def dist(a,b):
  return sum([int(c) for c in bin(a^b).split('b')[1]])

def bin_graph(n, k, trim = []):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    for t in trim:
      if dist(t,i)>k:
        flag = False
        break
    if flag:
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def small_graph(n, k, dists = [2,4,8]):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i) not in dists:
      continue
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def bin_graph1(n, k, trim = []):
  g = nx.Graph()
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    if dist(0,i)>k:
      flag = False
      
    if flag or i in set(trim):
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)


def r_graph(n, k):      # граф запретов,
  g = nx.Graph()        # ребрами соединены вершины на расстоянии >k
  for i in range(0,2**n):
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)>k):
        g.add_edge(i,j)
  return g

def write_metis(g0,fname):    # запись графа во входном формате KaMIS
  g = nx.convert_node_labels_to_integers(g0)
  with open(fname,"w") as f:
    f.write("{} {}\n".format(g.number_of_nodes(),g.number_of_edges()))
    for v in range(g.number_of_nodes()):
      f.write(" ".join([str(i+1) for i in sorted(list(g[v]))])+"\n")    

def write_cnf(g,ncolors,filename):  # запись КНФ для SAT-решателя    
  problem = ColSAT(g, ncolors)
  problem.build_cnf().to_file(filename)

def select_mis(g,filename):  # возвращает независимое множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==0:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      


def delete_mis(g,filename):  # удаляет независимосе множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==1:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      

# размерность, расстояние, номера вершин, по окрестностям которых 
# выполняется отсечение
 
r = bin_graph(10,6,trim=[0, 984, 730, 826, 1000, 718, 857])


#g = bin_graph(9,6,trim=[0,63])  
#print(r.number_of_nodes())
#print(r.number_of_edges())
#write_metis(r,"test.graph")

#!./redumis test.graph --output test.mis --time_limit=30

#g = r.copy()
g1 = r.copy()

print(g1.number_of_nodes())
print(g1.number_of_edges())

#write_cnf(g1,10,"test.cnf")

#!./kissat1 test.cnf

nmis = 0

283
15247


In [3]:
FILENAME = "test_11"
COLORSS = 0

def is_not_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return True

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if any(["UNSATISFIABLE" in line for line in outputt]):
         return True
    return False

def is_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return False

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if all(["UNSATISFIABLE" not in line for line in outputt]):
        if any(["SATISFIABLE" in line for line in outputt]):
            return True
    return False

def check_colorable_fast(graph, max_colors=100, timer=120):
    if graph.number_of_edges() == 0:
        return (None, 0)

    max_uncolorable = None
    min_colorable = None
    
    left_border = 1
    right_border = max_colors + 1
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_not_colorable(graph, color_count_to_check, timer):
            left_border = color_count_to_check
        else:
            right_border = color_count_to_check
    max_uncolorable = left_border

    left_border = 1
    right_border = max_colors
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_colorable(graph, color_count_to_check, timer):
            right_border = color_count_to_check
        else:
            left_border = color_count_to_check
    if right_border == max_colors:
        if is_colorable(graph, max_colors, timer):
            min_colorable = right_border 
        return (max_uncolorable, min_colorable)
    else:
        min_colorable = right_border
        return (max_uncolorable, min_colorable)
    return (None, None)

# my_graph = small_graph(10, 6)
# qq = my_graph.copy()
# check_colorable_fast(qq)

# Ниже рабочая зона (НЕТ)#

In [ ]:
# availiable #base_c #2, #4, #6_444, #6_24
# Варим множества, с которыми будет идти работа: перебираем дополнительный конф из #6_444, #6_24, имеем базовый конф в kv = ket vertexes, k_2 и k_4 просто добираем при варке графа
kv = [0, 945, 910, 630, 219, 365]
FILENAME = "NOK6_ISK5"
k_2 = []

for i in range(2**10):
    if dist(0, i) != 2:
        continue
    dst = [0, 0, 0, 0, 0, 0]
    ddst = 0
    flag = True
    for j in range(len(kv)):
        kvv = kv[j]
        if dist(i, kvv) > 6 or dist(i, kvv) == 0:
            flag = False
            break
        dst[j] = dist(i, kvv)
    if flag:
        k_2.append(i)

k_4 = []

for i in range(2**10):
    if dist(0, i) != 4:
        continue
    dst = [0, 0, 0, 0, 0, 0]
    ddst = 0
    flag = True
    for j in range(len(kv)):
        kvv = kv[j]
        if dist(i, kvv) > 6 or dist(i, kvv) == 0:
            flag = False
            break
        dst[j] = dist(i, kvv)
    if flag:
        k_4.append(i)


k6_24 = []

for i in range(2**10):
    if dist(0, i) != 6:
        continue
    dst = [0, 0, 0, 0, 0, 0]
    ddst = 0
    flag = True
    for j in range(len(kv)):
        kvv = kv[j]
        if dist(i, kvv) > 6 or dist(i, kvv) == 0:
            flag = False
            break
        dst[j] = dist(i, kvv)
    if flag and 2 in dst:
        k6_24.append(i)

k6_444 = []

for i in range(2**10):
    if dist(0, i) != 6:
        continue
    dst = [0, 0, 0, 0, 0, 0]
    ddst = 0
    flag = True
    for j in range(len(kv)):
        kvv = kv[j]
        if dist(i, kvv) > 6 or dist(i, kvv) == 0:
            flag = False
            break
        dst[j] = dist(i, kvv)
    if flag and 2 not in dst:
        k6_444.append(i)


In [ ]:
print(len(k_2), len(k_4), len(k6_24), len(k6_444))

NameError: ignored

In [ ]:
# Варим Граф, часть входных параметров не нужна.
def cook_graph_by_conf(base_conf, addition_conf, k_2, k_4, k6_24, k6_444, n=10, k=6):
    g = nx.Graph() 
    bans = base_conf + addition_conf
    pooles = k_2 + k_4 +  k6_24 + k6_444
    for i in bans:
        g.add_node(i)
    
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if flag:
            g.add_node(i)
    for i in list(g.nodes()):
        for j in g.nodes():
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

In [ ]:
k6_24[0], k6_444[0]

NameError: ignored

In [ ]:
# вот так примерно проверяются конфиги(базовый + основной), которые выпали при обработке
grph = bin_graph(10, 6, [0, 945, 910, 630, 219, 365, 63, 159, 187, 221, 189, 190])

is_colorable(grph, 11, 120)

In [ ]:
# вот такой функцией проверял, что будет, если в графе в итоге есть РОВНО + how_much вершин из тех, что находятся на расстоянии 6. 
# Т.е. чтобы заиспользовать код, нужно: убрать работу с k_2, k_4; передавать k6_24, k6_444 при варке графа

def bruteforce_nolimits_3(to_check, base_conf, add_conf, k_2, k_4, k6_24, k6_444, how_much=2, timer=120, n=10, k=6):
    ln = len(to_check)
    for i in range(ln):
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        if how_much > 1:
            bruteforce_nolimits_3(to_check[i+1:], base_conf, add_conf + [elem], k_2, k_4, to_check[i+1:], [], how_much-1, n, k)
        else:
            grph = cook_graph_by_conf(base_conf, add_conf + [elem], k_2, k_4, [], [], n, k)
            if not is_colorable(grph, 11, timer):
                print(base_conf, add_conf + [elem])

In [ ]:
bruteforce_nolimits_3(k6_24 + k6_444, [0, 945, 910, 630, 219, 365], [], k_2=k_2, k_4=k_4, k6_24=k6_24, k6_444=k6_444, how_much=1, timer=60)
bruteforce_nolimits_3(k6_24 + k6_444, [0, 945, 910, 630, 219, 365], [], k_2=k_2, k_4=k_4, k6_24=k6_24, k6_444=k6_444, how_much=2, timer=60)
bruteforce_nolimits_3(k6_24 + k6_444, [0, 945, 910, 630, 219, 365], [], k_2=k_2, k_4=k_4, k6_24=k6_24, k6_444=k6_444, how_much=3, timer=60)
bruteforce_nolimits_3(k6_24 + k6_444, [0, 945, 910, 630, 219, 365], [], k_2=k_2, k_4=k_4, k6_24=k6_24, k6_444=k6_444, how_much=4, timer=60)

In [ ]:
grph =cook_graph_by_conf([0, 945, 910, 630, 219, 365], [], k_2, k_4, [], [])
print(is_colorable(grph, 11, 120))

# Ниже рабочая зона (Да)#


In [4]:
FILENAME = "only_second_type"
kv = [0, 945, 910, 630]
tt = []
for i in range(1024):
    if dist(i, 0) % 2 == 1 or dist(i, 0) > 6 or i in kv:
        continue
    chk = 0
    for v in kv:
        if dist(v, i) > 6:
            chk = 10000
        elif dist(v, i) == 6:
            chk += 1
    if chk == 3:
        lhs = 0 ^ 945 ^ 910 ^ 630 ^ i
        flag = True
        for v in kv:
            if lhs ^ v == 0:
                flag = False
        if flag:
            tt.append(i)
print(len(tt))
tt

72


[3,
 5,
 24,
 27,
 29,
 40,
 43,
 45,
 159,
 175,
 187,
 189,
 192,
 195,
 197,
 215,
 216,
 231,
 232,
 246,
 250,
 252,
 287,
 303,
 315,
 317,
 320,
 323,
 325,
 343,
 344,
 359,
 360,
 374,
 378,
 380,
 414,
 430,
 435,
 437,
 467,
 469,
 474,
 476,
 483,
 485,
 490,
 492,
 599,
 603,
 605,
 615,
 619,
 621,
 634,
 636,
 655,
 697,
 715,
 717,
 729,
 745,
 783,
 825,
 843,
 845,
 857,
 873,
 970,
 972,
 977,
 993]

In [5]:
base_conf = [0, 945, 910, 630]

k2s = [(0, 945),
       (630, 910),
       (0, 910),
       (0, 630),
       (630, 945),
       (945, 910)]
kl3s = [(0, 630, 910),
        (0, 630, 945),
        (630, 910, 945),
        (0, 910, 945)]
kl4s = [(0, 945, 910, 630)]

k_2 = []
k_4 = []
k_6 = []

for i in range(2**10):
    if dist(0, i) % 2 == 1:
        continue
    flag = True
    for base_v in base_conf:
        if dist(i, base_v) == 0 or dist(i, base_v) > 6:
            flag = False
            continue
    big_flag = True
    for kl3 in kl3s:
        res = 0
        for v in kl3:
            res ^= v
        if res == i:
            big_flag = False
            break
    if not big_flag:
        continue
    for kl4 in kl4s:
        fflag = False
        for v in kl4:
            if dist(i, v) < 6:
                fflag = True
                break
        if not fflag:
            big_flag = False
            break
    if not big_flag:
        continue
    if flag:
        if dist(0, i) == 2:
            k_2.append(i)
        if dist(0, i) == 4:
            k_4.append(i)
        if dist(0, i) == 6:
            k_6.append(i)

In [6]:
BIG_NUM = 3
COUNTER = 0
BPM = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

def simple_cook_graph_no_K4_second(base_conf, add_conf, kl3s, kl4s, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
    if COUNTER % 50 == 0:
        print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue

        big_flag = True
        for kl3 in kl3s:
            res = 0
            for v in kl3:
                res ^= v
            if res == i:
                big_flag = False
                break
        for kl4 in kl4s:
            fflag = False
            for v in kl4:
                if dist(i, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if big_flag:
            g.add_node(i)
    for i in list(g.nodes()):
        for j in g.nodes():
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

def sort_to_check(to_check, base_conf, add_conf):
    to_to = [[a, 0] for a in to_check]
    for a in to_to:
        for b in base_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
        for b in  add_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
    tbd = sorted(to_to, key=lambda x: (-x[1], x[0]))
    return [a[0] for a in tbd]


def bruteforce_nolimits_nok4_second(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], kl4s=[], how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl3 in kl3s:
            res = 0
            for v in kl3:
                res ^= v
            if res == elem:
                big_flag = False
                break
        if not big_flag:
            continue
        for kl4 in kl4s:
            fflag = False
            for v in kl4:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl4s_updated = []
        for k3ss in kl3s:
            if dist(elem, k3ss[0]) == 6 and dist(elem, k3ss[1]) == 6 and dist(elem, k3ss[2]):
                kl4s_updated.append((k3ss[2], k3ss[0], k3ss[1], elem))
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        if how_much > 1:
            bruteforce_nolimits_nok4_second(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_no_K4_second(base_conf=base_conf, add_conf=add_conf + [elem], kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated, k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
    if how_much == BIG_NUM:
        print("analysing {} finished".format(BIG_NUM))

def bruteforce_nolimits_nok4_second_2(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], kl4s=[], how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    if how_much == BIG_NUM:
        ln = ln // 2
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl3 in kl3s:
            res = 0
            for v in kl3:
                res ^= v
            if res == elem:
                big_flag = False
                break
        if not big_flag:
            continue
        for kl4 in kl4s:
            fflag = False
            for v in kl4:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl4s_updated = []
        for k3ss in kl3s:
            if dist(elem, k3ss[0]) == 6 and dist(elem, k3ss[1]) == 6 and dist(elem, k3ss[2]):
                kl4s_updated.append((k3ss[2], k3ss[0], k3ss[1], elem))
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        if how_much > 1:
            bruteforce_nolimits_nok4_second(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_no_K4_second(base_conf=base_conf, add_conf=add_conf + [elem], kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated, k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
    if how_much == BIG_NUM:
        print("analysing {} finished".format(BIG_NUM))


def bruteforce_nolimits_nok4_second_short(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], kl4s=[], how_much=4, timer=120, n=10, k=6, fromm=0):
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl3 in kl3s:
            res = 0
            for v in kl3:
                res ^= v
            if res == elem:
                big_flag = False
                break
        if not big_flag:
            continue
        for kl4 in kl4s:
            fflag = False
            for v in kl4:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl4s_updated = []
        for k3ss in kl3s:
            if dist(elem, k3ss[0]) == 6 and dist(elem, k3ss[1]) == 6 and dist(elem, k3ss[2]):
                kl4s_updated.append((k3ss[2], k3ss[0], k3ss[1], elem))
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        grph = simple_cook_graph_no_K4_second(base_conf=base_conf, add_conf=add_conf + [elem], kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated, k_2=k_2, k_4=k_4, k_6=[], n=n, k=k)
        brb = is_colorable(grph, 11, timer)
        if not brb:
            if how_much == 1:
                print(base_conf, add_conf + [elem])
        if how_much > 1:
            bruteforce_nolimits_nok4_second_short(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated, kl4s = kl4s + kl4s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)

    if how_much == BIG_NUM:
        print("analysing {}-or-less-but-at-least-1 finished".format(BIG_NUM))

In [7]:
%%time
print("Before everything", COLORSS)
k_6 = sort_to_check(k_6, base_conf, [])
print('to check:', len(k_6[:60]), len(k_6))
# bruteforce_nolimits_nok4_second(k_6[:60], base_conf, [], k_2, k_4 + k_6[60:], k2s, kl3s, kl4s, 3, 20, 10, 6)
bruteforce_nolimits_nok4_second_2(k_6, base_conf, [], k_2, k_4, k2s, kl3s, kl4s, 3, 20, 10, 6)
print("After add 3", COLORSS)

Before everything 0
to check: 60 151
[[0, 0], [72, 149], [0, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [11, 148], [1, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [78, 148], [1, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [144, 148], [1, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [74, 147], [2, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [18, 146], [3, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [82, 146], [3, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [18, 141], [8, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [79, 141], [8, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [16, 140], [9, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [82, 140], [9, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [33, 139], [10, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [91, 139], [10, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [40, 138], [11, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [99, 138], [11, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [41, 137], [12, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [100, 137], [12, 150], [0, 75], [0, 0], [0, 0]]
[[0, 0], [38, 128], 

[[0, 0], [29, 136], [12, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [91, 136], [12, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [34, 127], [21, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [96, 127], [21, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [42, 126], [22, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [104, 126], [22, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [40, 125], [23, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [101, 125], [23, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [44, 124], [24, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [112, 124], [24, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [48, 123], [25, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [114, 123], [25, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [52, 122], [26, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [5, 121], [27, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [73, 121], [27, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [34, 120], [28, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [101, 120], [28, 149], [1, 75], [0, 0], [0, 0]]
[[0, 0], [75, 119], [29, 149], [1, 75], [0, 

[[0, 0], [13, 106], [41, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [82, 106], [41, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [42, 105], [42, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [13, 104], [43, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [74, 104], [43, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [34, 103], [44, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [1, 102], [45, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [65, 102], [45, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [26, 101], [46, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [90, 101], [46, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [58, 100], [47, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [15, 99], [48, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [75, 99], [48, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [57, 98], [49, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [22, 97], [50, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [91, 97], [50, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [54, 96], [51, 148], [2, 75], [0, 0], [0, 0]]
[[0, 0], [26, 95], [52, 148], [2, 75], [0, 0], [0, 0]]


[[0, 0], [60, 77], [69, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [45, 76], [70, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [37, 75], [71, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [26, 74], [72, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [15, 73], [73, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [7, 72], [74, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [69, 72], [74, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [63, 71], [75, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [59, 70], [76, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [60, 68], [78, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [62, 67], [79, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [59, 66], [80, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [55, 65], [81, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [51, 64], [82, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [53, 63], [83, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [56, 62], [84, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [57, 61], [85, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [1, 59], [87, 147], [3, 75], [0, 0], [0, 0]]
[[0, 0], [6,

[[0, 0], [22, 61], [84, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [29, 60], [85, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [30, 59], [86, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [37, 58], [87, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [47, 57], [88, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [0, 55], [90, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [9, 54], [91, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [22, 53], [92, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [33, 52], [93, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [47, 51], [94, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [11, 49], [96, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [37, 48], [97, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [13, 46], [99, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [34, 45], [100, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [12, 43], [102, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [41, 42], [103, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [30, 40], [105, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0], [25, 38], [107, 146], [4, 75], [0, 0], [0, 0]]
[[0, 0]

...

[[0, 0], [23, 58], [29, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [22, 57], [30, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [23, 56], [31, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [22, 52], [35, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [22, 51], [36, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [30, 48], [39, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [38, 47], [40, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [45, 46], [41, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [8, 44], [43, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [18, 43], [44, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [28, 42], [45, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [39, 41], [46, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [11, 39], [48, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [25, 38], [49, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [2, 36], [51, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [20, 33], [54, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [6, 31], [56, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [25, 30], [57, 88], [62, 75], [0, 0], [0, 0]]
[[0, 0], [16,

[[0, 0], [68, 81], [3, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [46, 80], [4, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [19, 79], [5, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [1, 78], [6, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [61, 78], [6, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [34, 77], [7, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [20, 76], [8, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [2, 75], [9, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [64, 75], [9, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [51, 74], [10, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [37, 73], [11, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [23, 72], [12, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [11, 71], [13, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [69, 71], [13, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [57, 70], [14, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [46, 69], [15, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [34, 68], [16, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [18, 67], [17, 85], [65, 75], [0, 0], [0, 0]]
[[0, 0], [13, 66], [1

[[0, 0], [47, 51], [31, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [3, 49], [33, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [8, 48], [34, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [16, 47], [35, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [22, 46], [36, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [34, 45], [37, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [2, 43], [39, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [19, 42], [40, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [35, 41], [41, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [14, 39], [43, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [35, 38], [44, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [17, 36], [46, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [5, 34], [48, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [27, 33], [49, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [18, 31], [51, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [16, 29], [53, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [20, 27], [55, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [6, 24], [58, 83], [67, 75], [0, 0], [0, 0]]
[[0, 0], [18, 2

[[0, 0], [46, 70], [9, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [34, 69], [10, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [19, 68], [11, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [10, 67], [12, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [2, 66], [13, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [56, 66], [13, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [45, 65], [14, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [38, 64], [15, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [28, 63], [16, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [21, 62], [17, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [15, 61], [18, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [10, 60], [19, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [6, 59], [20, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [3, 58], [21, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [2, 57], [22, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [1, 56], [23, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [1, 55], [24, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [0, 54], [25, 80], [70, 75], [0, 0], [0, 0]]
[[0, 0], [0, 53], 

[[0, 0], [64, 75], [1, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [40, 74], [2, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [26, 73], [3, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [11, 72], [4, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [69, 72], [4, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [55, 71], [5, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [35, 70], [6, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [22, 69], [7, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [12, 68], [8, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [4, 67], [9, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [60, 67], [9, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [51, 66], [10, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [40, 65], [11, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [34, 64], [12, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [28, 63], [13, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [20, 62], [14, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [17, 61], [15, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [13, 60], [16, 77], [73, 75], [0, 0], [0, 0]]
[[0, 0], [10, 59], [17

In [16]:
%%time
print("Before add 3, but last after first 75", COLORSS)
bruteforce_nolimits_nok4_second_short(k_6[75:], base_conf, [], k_2, k_4, k2s, kl3s, kl4s, 3, 20, 10, 6)
print("After add 3, but last after first 75", COLORSS)
bruteforce_nolimits_nok4_second_short(k_6, base_conf, [], k_2, k_4, k2s, kl3s, kl4s, 2, 20, 10, 6)
print("After add 2", COLORSS)
bruteforce_nolimits_nok4_second_short(k_6, base_conf, [], k_2, k_4, k2s, kl3s, kl4s, 1, 20, 10, 6)
print("After add 1", COLORSS)
# bruteforce_nolimits_nok4_second_short(k_6, base_conf, [], k_2, k_4, k2s, kl3s, kl4s, 0, 20, 10, 6)

Before add 3, but last after first 75 357035
[[0, 0], [12, 74], [0, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [70, 74], [0, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [47, 73], [1, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [30, 72], [2, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [17, 71], [3, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [2, 70], [4, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [61, 70], [4, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [44, 69], [5, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [29, 68], [6, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [17, 67], [7, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [8, 66], [8, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [1, 65], [9, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [60, 65], [9, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [51, 64], [10, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [41, 63], [11, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [31, 62], [12, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [26, 61], [13, 75], [0, 76], [0, 0], [0, 0]]
[[0, 0], [19, 60], [14, 75], [0, 76], [0, 0], [0, 0]]

[[0, 0], [30, 64], [7, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [20, 63], [8, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [8, 62], [9, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [1, 61], [10, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [56, 61], [10, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [46, 60], [11, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [39, 59], [12, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [31, 58], [13, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [25, 57], [14, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [23, 56], [15, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [20, 55], [16, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [17, 54], [17, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [16, 53], [18, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [14, 52], [19, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [15, 51], [20, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [19, 50], [21, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [20, 49], [22, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [25, 48], [23, 72], [3, 76], [0, 0], [0, 0]]
[[0, 0], [30, 47], [24, 72], [3, 

[[0, 0], [28, 42], [26, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [40, 41], [28, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [21, 39], [29, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [2, 35], [33, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [19, 34], [34, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [7, 32], [36, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [29, 31], [37, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [22, 29], [39, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [21, 27], [41, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [21, 24], [44, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [3, 21], [47, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [16, 19], [49, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [14, 15], [54, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [1, 10], [58, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [1, 2], [66, 69], [6, 76], [0, 0], [0, 0]]
[[0, 0], [52, 67], [0, 68], [7, 76], [0, 0], [0, 0]]
[[0, 0], [42, 66], [1, 68], [7, 76], [0, 0], [0, 0]]
[[0, 0], [29, 65], [2, 68], [7, 76], [0, 0], [0, 0]]
[[0, 0], [20, 64], [3, 68], [7, 76], 

[[0, 0], [12, 41], [23, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [25, 40], [24, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [3, 38], [26, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [22, 37], [27, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [3, 33], [31, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [27, 32], [32, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [18, 30], [34, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [11, 27], [37, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [12, 25], [39, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [16, 23], [41, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [1, 19], [45, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [15, 17], [47, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [3, 13], [51, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [8, 9], [56, 65], [10, 76], [0, 0], [0, 0]]
[[0, 0], [12, 63], [0, 64], [11, 76], [0, 0], [0, 0]]
[[0, 0], [8, 62], [1, 64], [11, 76], [0, 0], [0, 0]]
[[0, 0], [2, 61], [2, 64], [11, 76], [0, 0], [0, 0]]
[[0, 0], [57, 61], [2, 64], [11, 76], [0, 0], [0, 0]]
[[0, 0], [50, 60], [3,

[[0, 0], [30, 57], [2, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [29, 56], [3, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [27, 55], [4, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [31, 54], [5, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [33, 53], [6, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [37, 52], [7, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [41, 51], [8, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [46, 50], [9, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [2, 48], [11, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [9, 47], [12, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [19, 46], [13, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [30, 45], [14, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [42, 44], [15, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [16, 42], [17, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [30, 41], [18, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [3, 38], [21, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [22, 37], [22, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [3, 34], [25, 60], [15, 76], [0, 0], [0, 0]]
[[0, 0], [24, 33], [26

...

[[0, 0], [7, 29], [20, 50], [25, 76], [0, 0], [0, 0]]
[[0, 0], [1, 27], [22, 50], [25, 76], [0, 0], [0, 0]]
[[0, 0], [23, 26], [23, 50], [25, 76], [0, 0], [0, 0]]
[[0, 0], [20, 24], [25, 50], [25, 76], [0, 0], [0, 0]]
[[0, 0], [21, 22], [27, 50], [25, 76], [0, 0], [0, 0]]
[[0, 0], [5, 19], [30, 50], [25, 76], [0, 0], [0, 0]]
[[0, 0], [16, 17], [33, 50], [25, 76], [0, 0], [0, 0]]
[[0, 0], [1, 13], [36, 50], [25, 76], [0, 0], [0, 0]]
[[0, 0], [1, 9], [40, 50], [25, 76], [0, 0], [0, 0]]
[[0, 0], [1, 2], [48, 50], [25, 76], [0, 0], [0, 0]]
[[0, 0], [47, 48], [0, 49], [26, 76], [0, 0], [0, 0]]
[[0, 0], [2, 46], [2, 49], [26, 76], [0, 0], [0, 0]]
[[0, 0], [6, 45], [3, 49], [26, 76], [0, 0], [0, 0]]
[[0, 0], [11, 44], [4, 49], [26, 76], [0, 0], [0, 0]]
[[0, 0], [18, 43], [5, 49], [26, 76], [0, 0], [0, 0]]
[[0, 0], [24, 42], [6, 49], [26, 76], [0, 0], [0, 0]]
[[0, 0], [35, 41], [7, 49], [26, 76], [0, 0], [0, 0]]
[[0, 0], [6, 39], [9, 49], [26, 76], [0, 0], [0, 0]]
[[0, 0], [18, 38], [10, 49], 

[[0, 0], [22, 36], [5, 42], [33, 76], [0, 0], [0, 0]]
[[0, 0], [6, 34], [7, 42], [33, 76], [0, 0], [0, 0]]
[[0, 0], [27, 33], [8, 42], [33, 76], [0, 0], [0, 0]]
[[0, 0], [15, 31], [10, 42], [33, 76], [0, 0], [0, 0]]
[[0, 0], [5, 29], [12, 42], [33, 76], [0, 0], [0, 0]]
[[0, 0], [0, 27], [14, 42], [33, 76], [0, 0], [0, 0]]
[[0, 0], [25, 26], [16, 42], [33, 76], [0, 0], [0, 0]]
[[0, 0], [6, 23], [18, 42], [33, 76], [0, 0], [0, 0]]
[[0, 0], [12, 20], [21, 42], [33, 76], [0, 0], [0, 0]]
[[0, 0], [4, 17], [24, 42], [33, 76], [0, 0], [0, 0]]
[[0, 0], [8, 14], [27, 42], [33, 76], [0, 0], [0, 0]]
[[0, 0], [9, 10], [31, 42], [33, 76], [0, 0], [0, 0]]
[[0, 0], [2, 40], [0, 41], [34, 76], [0, 0], [0, 0]]
[[0, 0], [14, 39], [1, 41], [34, 76], [0, 0], [0, 0]]
[[0, 0], [27, 38], [2, 41], [34, 76], [0, 0], [0, 0]]
[[0, 0], [3, 36], [4, 41], [34, 76], [0, 0], [0, 0]]
[[0, 0], [21, 35], [5, 41], [34, 76], [0, 0], [0, 0]]
[[0, 0], [7, 33], [7, 41], [34, 76], [0, 0], [0, 0]]
[[0, 0], [28, 32], [8, 41], [

[[0, 0], [25, 28], [1, 30], [45, 76], [0, 0], [0, 0]]
[[0, 0], [20, 26], [3, 30], [45, 76], [0, 0], [0, 0]]
[[0, 0], [17, 24], [5, 30], [45, 76], [0, 0], [0, 0]]
[[0, 0], [18, 22], [7, 30], [45, 76], [0, 0], [0, 0]]
[[0, 0], [4, 19], [10, 30], [45, 76], [0, 0], [0, 0]]
[[0, 0], [16, 17], [13, 30], [45, 76], [0, 0], [0, 0]]
[[0, 0], [1, 13], [16, 30], [45, 76], [0, 0], [0, 0]]
[[0, 0], [1, 9], [20, 30], [45, 76], [0, 0], [0, 0]]
[[0, 0], [1, 2], [28, 30], [45, 76], [0, 0], [0, 0]]
[[0, 0], [19, 27], [1, 29], [46, 76], [0, 0], [0, 0]]
[[0, 0], [15, 25], [3, 29], [46, 76], [0, 0], [0, 0]]
[[0, 0], [14, 23], [5, 29], [46, 76], [0, 0], [0, 0]]
[[0, 0], [17, 21], [7, 29], [46, 76], [0, 0], [0, 0]]
[[0, 0], [6, 18], [10, 29], [46, 76], [0, 0], [0, 0]]
[[0, 0], [4, 15], [13, 29], [46, 76], [0, 0], [0, 0]]
[[0, 0], [9, 12], [16, 29], [46, 76], [0, 0], [0, 0]]
[[0, 0], [4, 7], [21, 29], [46, 76], [0, 0], [0, 0]]
[[0, 0], [18, 27], [0, 28], [47, 76], [0, 0], [0, 0]]
[[0, 0], [16, 25], [2, 28], [4

[[0, 0], [16, 126], [24, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [72, 126], [24, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [15, 125], [25, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [71, 125], [25, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [17, 124], [26, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [75, 124], [26, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [20, 123], [27, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [79, 123], [27, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [23, 122], [28, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [80, 122], [28, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [25, 121], [29, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [83, 121], [29, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [17, 120], [30, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [69, 120], [30, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [2, 119], [31, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [52, 119], [31, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [108, 119], [31, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [42, 118], [32, 151], 

[[0, 0], [9, 20], [130, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [0, 17], [133, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [14, 15], [136, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [7, 11], [139, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [0, 5], [145, 151], [75, 76], [0, 0], [0, 0]]
After add 2 432869
[[0, 0], [30, 151], [150, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [80, 151], [150, 151], [75, 76], [0, 0], [0, 0]]
[[0, 0], [130, 151], [150, 151], [75, 76], [0, 0], [0, 0]]
After add 1 433020
CPU times: user 15h 33min 24s, sys: 57min 47s, total: 16h 31min 11s
Wall time: 17h 52min 3s


In [14]:
%%time
grph = simple_cook_graph_no_K4_second(base_conf, [], kl3s, kl4s, k_2, k_4, [], n=10, k=6)
brb = is_colorable(grph, 11, 20)
brb

CPU times: user 1.34 s, sys: 33.8 ms, total: 1.38 s
Wall time: 1.49 s


True

In [10]:
print("we win")

we win


In [ ]:
print(0b0111111000, 0b0111000111, 0b0000111111, 0b1010010010)


504 455 63 658


0b11110001001000000
